In [1]:
from scipy.spatial import KDTree
from geopy.distance import geodesic
import pandas as pd

### DATA SOURCE: https://data.ibb.gov.tr/en/dataset/akaryakit-istasyonlari/resource/5625860c-d79a-446f-898e-2aa2b9099bc8

In [18]:
gh_df = (
    pd.read_csv("datasets/15_gh_info/geohash_traffic_density_pt_15clusters.csv", usecols=['GEOHASH', 'LATITUDE', 'LONGITUDE'])
)

gh_df.head()

,GEOHASH,LATITUDE,LONGITUDE
0,sxk3xw,41.080627,28.811646
1,sxk9nm,40.987244,29.108276
2,sxk9q0,41.003723,29.097290
3,sxk3hx,40.998230,28.679810
4,sx7cmx,41.042175,28.020630


In [16]:
fuel_stns = (
    pd.read_csv("datasets/16_tr_ist_fuel_stations/fuel_station.csv", usecols=['longtitude', 'latitude'])
    .assign(id=lambda x: 'id_' + x.reset_index()['index'].astype('string'))
    .reindex(columns=['id', 'latitude', 'longtitude'])
)

fuel_stns.head()

,id,latitude,longtitude
0,id_0,41.014146,29.163734
1,id_1,41.155796,29.604337
2,id_2,41.236515,28.906075
3,id_3,41.027432,28.847472
4,id_4,40.931624,29.338903


In [23]:
site_dict = (
    fuel_stns
    .set_index('id').T
    .to_dict()
)

In [24]:
# Unique list of coordinates
coordinates = gh_df[['LATITUDE', 'LONGITUDE']].drop_duplicates().values
coordinates

array([[41.080627, 28.811646],
       [40.987244, 29.108276],
       [41.003723, 29.09729 ],
       ...,
       [40.976257, 29.229126],
       [41.009216, 27.98767 ],
       [41.020203, 27.998657]])

In [25]:
def find_nearest_coordinates(
        target_lat: float,
        target_lon: float,
        coordinates: list[list[float, float]]
        ) -> list[tuple[tuple[float, float], float]]:
    """
    Finds the nearest coordinates to a target latitude and longitude.

    Args:
        target_lat (float): The latitude of the target location.
        target_lon (float): The longitude of the target location.
        coordinates (List[Tuple[float, float]]): A list of tuples representing the coordinates to search from.

    Returns:
        List[Tuple[Tuple[float, float], float]]: A list of tuples containing the nearest coordinates and their distances in kilometers.
    """
    target_coords = (target_lat, target_lon)
    
    # Build a KDTree for efficient nearest neighbor search
    tree = KDTree(coordinates)

    # Calculate the search radius in degrees
    radius_deg = 1 / 111.0  # 1 km radius in degrees

    # Query the KDTree for indices of points within the search radius
    indices = tree.query_ball_point([(target_lat, target_lon)], radius_deg)

    # Calculate the distance between target coordinates and nearest coordinates
    nearest_coordinates = [(coordinates[i], geodesic(target_coords, coordinates[i]).km) for i in indices[0]]

    # Sort the nearest coordinates by distance
    nearest_coordinates.sort(key=lambda x: x[1])  # Sort by distance

    return nearest_coordinates

In [27]:
fuelstn_list = []
new_list = []

for site in site_dict:

    target_lat = site_dict[site]['latitude']
    target_lon = site_dict[site]['longtitude']

    nearest_coords = find_nearest_coordinates(target_lat, target_lon, coordinates)

    for i, (coords, distance) in enumerate(nearest_coords):

        gh = gh_df.loc[gh_df['LATITUDE'].eq(coords[0]) & gh_df['LONGITUDE'].eq(coords[1]), 'GEOHASH'].values[0]

        # Creating two dataframe which will later be merged rows wise for visualization purpose.
        fuelstn_list.append({
            "type" : "fuel_stn",
            "name" : site,
            "latitude" : target_lat,
            "longitude" : target_lon,
            "nearest" : site
        })

        new_list.append({
            "type" : "geohash",
            "name" : gh,
            "latitude" : coords[0],
            "longitude" : coords[1],
            "nearest" : site
        })

In [28]:
df = (
    pd.concat([pd.DataFrame(new_list), pd.DataFrame(fuelstn_list)], ignore_index=True)
    .drop_duplicates(ignore_index=True)
)

df

,type,name,latitude,longitude,nearest
0,geohash,sxk9rd,41.014710,29.163208,id_0
1,geohash,sxk9r9,41.009216,29.163208,id_0
2,geohash,sxk9re,41.020203,29.163208,id_0
3,geohash,sxkd9f,41.234436,28.910522,id_2
4,geohash,sxkd9d,41.234436,28.899536,id_2
...,...,...,...,...,...
2902,fuel_stn,id_719,40.993986,28.894570,id_719
2903,fuel_stn,id_720,40.945854,29.127442,id_720
2904,fuel_stn,id_721,40.996974,29.088793,id_721
2905,fuel_stn,id_722,41.005599,29.146205,id_722


In [29]:
# Some GEOHASH has up to 10 fuel stations within its 1km radius
(
    df.query("type=='geohash'")
    .groupby('name')['nearest'].nunique()
    .nlargest(10)
)

name
sxk99t    10
sxk981     9
sxk99b     9
sxk984     8
sxk99s     8
sxk99w     8
sxkb96     8
sxkb97     8
sxkc23     8
sxk377     7
Name: nearest, dtype: int64

In [30]:
df.to_csv("datasets/13_gh_proximities/gh_nearby_fuelstns.csv", index=False)